In [22]:
import geopandas as gpd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import r2_score

# Cargar el GeoDataFrame
gdf = gpd.read_file(filename="Data/ECa.gpkg",layer="EC_field_04")

# Extraer coordenadas
coords = gdf.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()

# Calcular la matriz de distancias
distances = np.zeros((len(coords), len(coords)))
for i, coord1 in enumerate(coords):
    for j, coord2 in enumerate(coords):
        distances[i, j] = np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Extraer los valores a predecir
values = gdf['EC30'].values

# Definir los modelos de tendencia espacial a evaluar
models = {
    'constante': np.ones(len(coords)),
    '~x+y': [coord[0]+coord[1] for coord in coords],
    '~x+y+x*y': [coord[0]+coord[1]+coord[0]*coord[1] for coord in coords],
    '~x²+y²': [coord[0]**2+coord[1]**2 for coord in coords],
    '~x+y+x²+y²': [coord[0]+coord[1]+coord[0]**2+coord[1]**2 for coord in coords],
    '~x²+y²+x*y': [coord[0]**2+coord[1]**2+coord[0]*coord[1] for coord in coords],
    '~x+y+x²+y²+x*y': [coord[0]+coord[1]+coord[0]**2+coord[1]**2+coord[0]*coord[1] for coord in coords]
}
# Inicializar variables para almacenar el mejor modelo y su R² ajustado
best_model = None
best_r2_adjusted = -np.inf
best_name = None
# Evaluar cada modelo
for name, trend in models.items():
    X = np.column_stack((trend, [coord[1] for coord in coords]))  # Ajustar modelo con coordenada y
    X = sm.add_constant(X)  # Añadir constante al modelo
    model = sm.OLS(values, X)
    results = model.fit()
    trend_pred = results.predict(X)
    r2 = r2_score(values, trend_pred)
    n = len(values)  # número de observaciones
    k = X.shape[1] - 1  # número de predictores en el modelo (excluyendo la constante)
    r2_adjusted = 1 - ((1 - r2) * (n - 1) / (n - k - 1))
    print(f'Model: {name}, adj_R²: {round(r2_adjusted,4)}')
    if r2_adjusted > best_r2_adjusted:
        best_name = name
        best_r2_adjusted = r2_adjusted
        best_model = results
# Imprimir el mejor modelo y su R² ajustado
print(f'...\nBest model: {best_name}, adj_R²: {round(best_r2_adjusted,4)}')

In [ ]:
import geopandas as gpd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import r2_score

# Cargar el GeoDataFrame
gdf = gpd.read_file(filename="Data/ECa.gpkg", layer="EC_field_04")

# Extraer coordenadas
coords = gdf.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()

# Calcular la matriz de distancias
distances = np.zeros((len(coords), len(coords)))
for i, coord1 in enumerate(coords):
    for j, coord2 in enumerate(coords):
        distances[i, j] = np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Extraer los valores a predecir
values = gdf['EC30'].values


In [27]:

# Definir los modelos de covarianza espacial a evaluar
covariance_models = {
    'Gau': np.exp(-distances),  # Modelo de covarianza gaussiana
    'Sph': 1 - (3/2) * distances + (1/2) * (distances ** 3),  # Modelo de covarianza esférica
    'Exp': np.exp(-distances),  # Modelo de covarianza exponencial
    'Ste': distances * np.exp(-distances)  # Modelo de covarianza estable
}

# Inicializar variables para almacenar el mejor modelo de tendencia, modelo de covarianza y sus métricas
best_trend_model = None
best_covariance_model = None
best_r2 = -np.inf
best_bic = np.inf
best_trend_name = None
best_covariance_name = None

# Evaluar cada modelo de covarianza
for cov_name, covariance in covariance_models.items():
    # Definir los modelos de tendencia espacial a evaluar
    models = {
        'constante': np.ones(len(coords)),
        '~x+y': [coord[0] + coord[1] for coord in coords],
        '~x+y+x*y': [coord[0] + coord[1] + coord[0] * coord[1] for coord in coords],
        '~x²+y²': [coord[0]**2 + coord[1]**2 for coord in coords],
        '~x+y+x²+y²': [coord[0] + coord[1] + coord[0]**2 + coord[1]**2 for coord in coords],
        '~x²+y²+x*y': [coord[0]**2 + coord[1]**2 + coord[0] * coord[1] for coord in coords],
        '~x+y+x²+y²+x*y': [coord[0] + coord[1] + coord[0]**2 + coord[1]**2 + coord[0] * coord[1] for coord in coords]
    }

    # Evaluar cada modelo de tendencia espacial
    for trend_name, trend in models.items():
        X = np.column_stack((trend, [coord[1] for coord in coords]))  # Ajustar modelo con coordenada y
        X = sm.add_constant(X)  # Añadir constante al modelo
        model = sm.OLS(values, X)
        results = model.fit()
        trend_pred = results.predict(X)
        r2 = r2_score(values, trend_pred)
        n = len(values)  # número de observaciones
        k = X.shape[1] - 1  # número de predictores en el modelo (excluyendo la constante)
        r2_adjusted = 1 - ((1 - r2) * (n - 1) / (n - k - 1))
        bic = results.bic
        print(f'Modelo de tendencia: {trend_name}, Modelo de covarianza: {cov_name}, adj_R²: {round(r2_adjusted, 4)}, BIC: {round(bic, 4)}')
        if r2_adjusted > best_r2:
            best_r2 = r2_adjusted
            best_trend_model = results
            best_trend_name = trend_name
        if bic < best_bic:
            best_bic = bic
            best_covariance_model = covariance
            best_covariance_name = cov_name

# Imprimir el mejor modelo de tendencia y modelo de covarianza
print(f'...\nMejor modelo de tendencia:{best_trend_name} adj_R²: {round(best_r2, 4)}')
print(f'Mejor modelo de covarianza:{best_covariance_name} BIC: {round(best_bic, 4)}')


Modelo de tendencia: constante, Modelo de covarianza: Gau, adj_R²: 0.0004, BIC: 18011.261
Modelo de tendencia: ~x+y, Modelo de covarianza: Gau, adj_R²: 0.182, BIC: 17408.9737
Modelo de tendencia: ~x+y+x*y, Modelo de covarianza: Gau, adj_R²: 0.1819, BIC: 17401.2265
Modelo de tendencia: ~x²+y², Modelo de covarianza: Gau, adj_R²: 0.0043, BIC: 17998.4055
Modelo de tendencia: ~x+y+x²+y², Modelo de covarianza: Gau, adj_R²: 0.0043, BIC: 17998.4055
Modelo de tendencia: ~x²+y²+x*y, Modelo de covarianza: Gau, adj_R²: 0.1025, BIC: 17683.0423
Modelo de tendencia: ~x+y+x²+y²+x*y, Modelo de covarianza: Gau, adj_R²: 0.1025, BIC: 17683.0422
Modelo de tendencia: constante, Modelo de covarianza: Sph, adj_R²: 0.0004, BIC: 18011.261
Modelo de tendencia: ~x+y, Modelo de covarianza: Sph, adj_R²: 0.182, BIC: 17408.9737
Modelo de tendencia: ~x+y+x*y, Modelo de covarianza: Sph, adj_R²: 0.1819, BIC: 17401.2265
Modelo de tendencia: ~x²+y², Modelo de covarianza: Sph, adj_R²: 0.0043, BIC: 17998.4055
Modelo de tend

In [ ]:

# Obtener la tendencia espacial del mejor modelo
best_trend = best_model.predict()

# Calcular los residuos
residuals = values - best_trend

# Crear la instancia de OrdinaryKriging solo para los residuos
ok = OrdinaryKriging(
    coords,  # Coordenadas
    residuals,  # Residuos
    variogram_model='linear',  # Modelo de variograma
    nlags=10  # Número de lags
)

# Obtener la malla de predicción
gridx = np.linspace(gdf.bounds.minx.min(), gdf.bounds.maxx.max(), 100)
gridy = np.linspace(gdf.bounds.miny.min(), gdf.bounds.maxy.max(), 100)

# Realizar la predicción solo para los residuos
z, ss = ok.estimate(
    gridx,
    gridy,
    backend='vectorized'
)

# Sumar los residuos predichos y la tendencia para obtener la predicción final
prediction = z + trend.mean()
